In [ ]:
from io import BytesIO
import json

import requests
from matplotlib import pyplot as plt
import matplotlib.patches as patches

Больше подробностей в документации: 
- https://mcs.mail.ru/docs/ru/ml/vision/manage-vision/vision-api-obj
- https://mcs.mail.ru/docs/ru/additionals/api/vision-api

In [ ]:
# определение эндпойнтов
HOST = "https://smarty.mail.ru"

URL_OBJ_DETECT = HOST + "/api/v1/objects/detect"
URL_FR_SET = HOST + "/api/v1/persons/set"
URL_RECOGNIZE = HOST + "/api/v1/persons/recognize"

In [ ]:
# подготовка параметров для авторизации запросов
oauth_provider = "mcs"
oauth_token_path = "./token"
with open(oauth_token_path, "r") as f:
    oauth_token = f.read()

# для пробрасывания авторизации в URL
authorization_params = {
    "oauth_provider": oauth_provider, 
    "oauth_token": oauth_token
}

In [ ]:
def plot_image(image, figsize=(15, 15)):
    """Функция для отрисовки изображений внутри ноутбучка"""
    plt.figure(figsize=figsize)
    plt.imshow(image, interpolation="lanczos")
    plt.axis('off')

# Распознавание авто

**Сценарий**
1. с некоторой периодичностью кадр из видеопотока отправляется на распознавание наличия авто
2. если в кадре дейстувительно обнаружен автомобиль, то
3. кадр отправляется на распознавание номеров
4. если распознанный номер входит в число валидных, то проезд открывается

In [ ]:
def request_obj_detect(URL: str, img: bytes, mode: list[str]):
    """Функция обертка для упрощенной отправки запроса к эндпойнту objects/detect"""
    # создать словарь с файлами для параметра files в post запросе
    files = {}
    name = "file_0"
    files[name] = img
    
    # создать meta: Параметры запроса передаются в формате JSON в теле запроса с name="meta"
    meta = {
        "mode": mode,
        "images": [{"name":name}]
    }
    data = {'meta': json.dumps(meta)}
    
    response = requests.post(URL, params=authorization_params, data=data, files=files)
    return response

In [ ]:
def parse_obj_detection_result(response: requests.models.Response) -> list:
    """
    Парсинг респонса с найденными объектами. Возвращает dict для удобной подачи в функцию отрисовки bounding boxes
    
    RETURN
    bounding_boxes: {
        "label_1": [left x, top y, right x, bottom y],
        ...
        "label_n": [left x, top y, right x, bottom y]
    }
    """
    
    body = response.json()["body"]
    response_type = list(body.keys())[0]
    recognized_objects = body[response_type][0]["labels"]
    bb_dict = []
    for i, box_info in enumerate(recognized_objects, 0):
        label = box_info['eng']
        coord = box_info['coord']
        bb_dict.append(coord)
    return bb_dict

def parse_coords_for_rectangle(coord: list[int]):
    """
    Необходимо вернуть в удобном для отрисовке в patches.Rectangle виде
    На вход подается список из 4 точек: [left x, top y, right x, bottom y]
    Также учесть инверсию оси Y изображения в API Vision и в matplotlib. Vision считает сверху, matplotlib - снизу
    """
    
    width = coord[2] - coord[0]
    height = coord[1] - coord[3]  # из-за инверсии оси Y 
    xy = (coord[0], coord[3])
    return xy, width, height

def plot_image_with_bb(image, bounding_boxes: list, bb_color: str='blue'):
    """
    Отрисовка bounding box
    """
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(image, interpolation="lanczos")
    
    for coord in bounding_boxes:
        xy, width, height = parse_coords_for_rectangle(coord)
        rect = patches.Rectangle(xy, width, height, linewidth=2, edgecolor=bb_color, facecolor='none')
        ax.add_patch(rect)
    plt.show()

In [ ]:
# база номеров, которым въезд разрешен
VALID_NUMBERS = [
    'А212АА99',
    '   ...   ',
    'У329СО197'
]

# название класса автомобиля в ответе от модели
CAR_TAG = 'Car'

## Наглядно по шагам

### В кадре нет автомобиля

In [ ]:
# укажите путь до фотографии 
image_path = "./photo_for_demo/car_number/car_number_0_empty.jpg"
with open(image_path, "rb") as image:
    image_raw = image.read()
image = plt.imread(image_path)
plot_image(image, figsize=(11, 11))

In [ ]:
response = request_obj_detect(URL_OBJ_DETECT, img=image_raw, mode=['multiobject'])
response.json()

**Автомобиля нет, далее работа по распознаванию не ведется**

### В кадре есть автомобиль

In [ ]:
# укажите путь до фотографии 
image_path = "./photo_for_demo/car_number/car_number_1.jpg"
with open(image_path, "rb") as image:
    image_raw = image.read()
image = plt.imread(image_path)
plot_image(image)

In [ ]:
response = request_obj_detect(URL_OBJ_DETECT, img=image_raw, mode=['multiobject'])
response.json()

In [ ]:
bounding_boxes = parse_obj_detection_result(response)
plot_image_with_bb(image, bounding_boxes)

**Автомобиль обнаружен, далее отправляем на распознавание номеров**

In [ ]:
response = request_obj_detect(URL_OBJ_DETECT, img=image_raw, mode=['car_number'])
response.json()

In [ ]:
bounding_boxes = parse_obj_detection_result(response)
plot_image_with_bb(image, bounding_boxes)

In [ ]:
car_number = response.json()['body']['car_number_labels'][0]['labels'][0]['rus']
if car_number in VALID_NUMBERS:
    print('OPEN THE DOOR!')
else:
    print('HOLD THE DOOR!')

## Сборка решения

In [ ]:
def detect_auto(image_raw: bytes):
    """Проверка на наличие автомобиля в кадре"""
    response = request_obj_detect(URL_OBJ_DETECT, img=image_raw, mode=['multiobject'])
    recognized_objects = response.json()['body']['multiobject_labels'][0]['labels']
    
    for obj in recognized_objects:
        if obj['eng'] == CAR_TAG:
            return True
    return False


def recognize_car_number(image_raw: bytes):
    """Распознавание номера"""
    response = request_obj_detect(URL_OBJ_DETECT, img=image_raw, mode=['car_number'])
    car_number = response.json()['body']['car_number_labels'][0]['labels'][0]['rus']
    return car_number


def run_pipeline(image_path: str):
    with open(image_path, "rb") as image:
        image_raw = image.read()
    
    if detect_auto(image_raw):
        car_number = recognize_car_number(image_raw)
        print(car_number)
        if car_number in VALID_NUMBERS:
            print('OPEN THE DOOR!')
        else:
            print('HOLD THE DOOR!')
    else:
        print('HOLD THE DOOR!')

In [ ]:
image_path = "./photo_for_demo/car_number/car_number_1.jpg"
image = plt.imread(image_path)
plot_image(image, figsize=(15, 15))

In [ ]:
run_pipeline(image_path)

In [ ]:
image_path = "./photo_for_demo/car_number/car_number_2.jpg"
image = plt.imread(image_path)
plot_image(image, figsize=(15, 15))

In [ ]:
VALID_NUMBERS

In [ ]:
run_pipeline(image_path)

# Распознавание лиц

Для распознавания лиц используются четыре метода API:
- **set**:       позволяет установить связь между заданной фотографией и конкретным  person_id
- **recognize**: позволяет распознать person по заданной фотографии
- **delete**:    позволяет удалить связь между фотографией и person_id
- **truncate**:  позволяет полностью очистить space от внесенных person_id

В этом разделе демо, используя метод set, зададим определенный person_id для человека с фотографии. Затем с помощью метода recognize распознаем этого человека на других фотографиях.

Больше подробностей в документации: https://mcs.mail.ru/docs/ru/ml/vision/manage-vision/face-recognition

In [ ]:
NAMESPACE_FOR_DEMO = '7'
VALID_PERSONS = {}

**Шаг 1: у нас должна быть база валидных лиц, которых мы пропускаем на территорию**

In [ ]:
def request_set(URL: str, img: bytes, person_id: int, space: str="0"):
    """Функция-обёртка для обращения к методу set"""
    
    # создать словарь с файлами для параметра files в post запросе
    files = {}
    name = "file_0"
    files[name] = img
    
    # создать meta: Параметры запроса передаются в формате JSON в теле запроса с name="meta"
    meta = {
        "space": space,
        "images": [{"name":name, "person_id": person_id}]
    }
    data = {'meta': json.dumps(meta)}
    
    # отправка запроса
    response = requests.post(URL, params=authorization_params, data=data, files=files)
    return response

In [ ]:
def request_recognize(URL: str, img: bytes, create_new: bool=False, space: str="0"):
    """Функция обертка для упрощенной отправки запроса к эндпойнту recognize"""
    
    # создать словарь с файлами для параметра files в post запросе
    files = {}
    name = "file_0"
    files[name] = img
    
    # создать meta: Параметры запроса передаются в формате JSON в теле запроса с name="meta"
    meta = {
        "space": space,
        "create_new": create_new,
        "images": [{"name":name}]
    }
    data = {'meta': json.dumps(meta)}
    
    # отправка запроса
    response = requests.post(URL, params=authorization_params, data=data, files=files)
    return response

In [ ]:
def parse_persons_response(response):
    persons_list = response.json()['body']['objects'][0]['persons']
    persons_bb = []

    for person in persons_list:
        persons_bb.append(person['coord'])
    return persons_bb

**Присвоить person_id определенному человеку**

In [ ]:
# укажите путь до фотографии 
image_path_set = "./photo_for_demo/fr/fr_1_dim_set.jpg"
# укажите, какой person_id хотите присвоить этому человеку
person_id = 2

In [ ]:
with open(image_path_set, "rb") as image:
    image_set_raw = image.read()
image_set = plt.imread(image_path_set)
plot_image(image_set, figsize=(7, 7))

In [ ]:
# поставим в соответствие человеку с фотографии person_id
response = request_set(URL_FR_SET, img=image_set_raw, person_id=person_id, space=NAMESPACE_FOR_DEMO)
response.json()

In [ ]:
# дополним список валидных персон, которым разрешен проход на территорию
if response.json()['status'] == 200:
    VALID_PERSONS[f'person{person_id}'] = 'Димитрий'

In [ ]:
VALID_PERSONS

**Распознавание на проходной**

**Человек из белого списка**

In [ ]:
# укажите путь до фотографии
image_path_recognize = "./photo_for_demo/fr/fr_1_dim_rec1.jpg"

with open(image_path_recognize, "rb") as image:
    image_recognize_raw = image.read()
image_recognize = plt.imread(image_path_recognize)
plot_image(image_recognize, figsize=(7, 7))

In [ ]:
response = request_recognize(URL_RECOGNIZE, img=image_recognize_raw, create_new=False, space=NAMESPACE_FOR_DEMO)
response.json()

In [ ]:
plot_image_with_bb(image_recognize, parse_persons_response(response), bb_color='g')

**Человек не из белого списка**

In [ ]:
# укажите путь до фотографии
image_path_recognize = "./photo_for_demo/fr/fr_2_alex_rec.png"

with open(image_path_recognize, "rb") as image:
    image_recognize_raw = image.read()
image_recognize = plt.imread(image_path_recognize)
plot_image(image_recognize, figsize=(7, 7))

In [ ]:
response = request_recognize(URL_RECOGNIZE, img=image_recognize_raw, create_new=False, space=NAMESPACE_FOR_DEMO)
response.json()

In [ ]:
plot_image_with_bb(image_recognize, parse_persons_response(response), bb_color='r')